In [109]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize

In [106]:
initial_dataset = pd.read_csv("biodegradable_a.csv")
total_len, _ = initial_dataset.shape

# NOTE - NO INDEPENDENT VALIDATION SET !!!

# Total with means
#categorical = ['int16', 'int32', 'int64']
#biodegradable = ['object']
#numerical = ['float16', 'float32', 'float64']

class_cols = [col for col in initial_dataset.drop("Biodegradable", axis=1) if initial_dataset[col].apply(lambda x: x % 1 == 0).all()]
num_cols = [col for col in initial_dataset.drop("Biodegradable", axis=1) if initial_dataset[col].apply(lambda x: x % 1 != 0).any()]

print(len(class_cols) + len(num_cols))
print(len(initial_dataset.drop("Biodegradable", axis=1).columns))

total_categorical_dataset = initial_dataset[class_cols]
#total_categorical_dataset = total_categorical_dataset.fillna(total_categorical_dataset.mode())
total_categorical_dataset = total_categorical_dataset.fillna(-1)
total_categorical_dataset = total_categorical_dataset.astype(int).astype(object)
#print(total_categorical_dataset)

total_numerical_dataset = initial_dataset[num_cols]
total_numerical_dataset = total_numerical_dataset.fillna(total_numerical_dataset.mean())

total_biodegradable = initial_dataset["Biodegradable"]
#total_biodegradable = initial_dataset.select_dtypes(include=biodegradable)
total_biodegradable = total_biodegradable.fillna("")

# Scale numerical data
# https://scikit-learn.org/stable/modules/preproce
#print(total_numerical_dataset)

scaler = StandardScaler()
#scaler = MinMaxScaler(feature_range=(-1, 1))

total_numerical_dataset = pd.DataFrame(scaler.fit_transform(total_numerical_dataset),
             columns=total_numerical_dataset.columns, index=total_numerical_dataset.index)


#total_numerical_dataset = pd.DataFrame(normalize(total_numerical_dataset, norm='l2', axis=1, copy=True, return_norm=False),
#             columns=total_numerical_dataset.columns, index=total_numerical_dataset.index)

#print(total_numerical_dataset)
#
total_dataset = pd.concat([total_categorical_dataset, total_numerical_dataset,total_biodegradable], axis=1)
print(total_dataset)

total_len, _ = total_dataset.shape
train_dataset_len = round(total_len * 0.75)

dataset_train = total_dataset[0:train_dataset_len]
dataset_test = total_dataset[train_dataset_len:total_len]

print(total_dataset.shape)

# Removal of None/NaN vals
dropna_dataset = initial_dataset.dropna()

dropna_len, _ = dropna_dataset.shape
model_dropna_len = round(total_len * 0.75)

dropna_train = dropna_dataset[0:model_dropna_len]
dropna_test = dropna_dataset[model_dropna_len:dropna_len]

print(dropna_dataset.shape)

41
41
     nHM F04 NssssC nCb nO F03 nN_N nArNO2 nCRX3 B01  ...       SpMax_A  \
0      0   0      0   0  0   0    0      0     0   0  ... -9.976466e-01   
1      0   0      0   0  1   0    0      0     0   0  ...  9.352975e-15   
2      0   0      0   0  4   0    0      0     0   0  ...  9.352975e-15   
3      0   0      0   0  2   0    0      0     0   0  ... -3.725050e+00   
4      0   0      0   0  4   0    0      0     0   0  ... -7.185880e-01   
...   ..  ..    ...  .. ..  ..  ...    ...   ...  ..  ...           ...   
4559   0   0      0   0  2   0    0      0     0   0  ...  9.352975e-15   
4560   0   0      0   0  0   0    0      0     0   0  ... -1.946549e+00   
4561   0   0      0   0  2   0    0      0     0   0  ...  9.352975e-15   
4562   0   0      0   0  3   0    0      0     0   0  ... -2.646564e-01   
4563   0   0      0   0  1   0    0      0     0   0  ... -1.409268e+00   

      Psi_i_1d           SdO     TI2_L          nCrt       SpMax_B   Psi_i_A  \
0     0.11632

In [93]:
print(total_dataset.columns)
print(dropna_test.columns)

Index(['nHM', 'F04', 'NssssC', 'nCb', 'nO', 'F03', 'nN_N', 'nArNO2', 'nCRX3',
       'B01', 'B03', 'N_073', 'B04', 'C_026', 'F02_CN', 'nHDon', 'nN',
       'nArCOOR', 'SM6_L', 'Mi', 'SpPosA_B', 'Biodegradable'],
      dtype='object')
Index(['SpMax_L', 'J_Dz(e)', 'nHM', 'F01', 'F04', 'NssssC', 'nCb', 'C', 'nCp',
       'nO', 'F03', 'SdssC', 'HyWi_B', 'LOC', 'SM6_L', 'F03_CO', 'Me', 'Mi',
       'nN_N', 'nArNO2', 'nCRX3', 'SpPosA_B', 'nCIR', 'B01', 'B03', 'N_073',
       'SpMax_A', 'Psi_i_1d', 'B04', 'SdO', 'TI2_L', 'nCrt', 'C_026', 'F02_CN',
       'nHDon', 'SpMax_B', 'Psi_i_A', 'nN', 'SM6_B', 'nArCOOR', 'nX',
       'Biodegradable'],
      dtype='object')


## Using Model with replaced values when NaN, and discarding the dropped NaN values dataset

In [94]:
X_train = dataset_train.drop(["Biodegradable"], axis=1)
y_train = dataset_train.Biodegradable
print(X_train)
print(y_train)

      nHM  F04  NssssC  nCb   nO  F03  nN_N  nArNO2  nCRX3  B01  ...  N_073  \
0     0.0  0.0     0.0  0.0  0.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
1     0.0  0.0     0.0  0.0  1.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
2     0.0  0.0     0.0  0.0  4.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
3     0.0  0.0     0.0  0.0  2.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
4     0.0  0.0     0.0  0.0  4.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
...   ...  ...     ...  ...  ...  ...   ...     ...    ...  ...  ...    ...   
3418  0.0  0.0     0.0  2.0  2.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
3419  0.0  0.0     0.0  3.0  3.0  2.0   0.0     0.0    0.0  0.0  ...    0.0   
3420  0.0  0.0     0.0  2.0  2.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
3421  0.0  0.0     0.0  0.0  2.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   
3422  0.0  0.0     0.0  0.0  1.0  0.0   0.0     0.0    0.0  0.0  ...    0.0   

      B04  C_026  F02_CN  nHDon   nN  nArCOOR     S

In [95]:
X_test = dataset_test.drop(["Biodegradable"], axis=1)
y_test = dataset_test.Biodegradable

## Testing Random Forests for Feature Selection

In [110]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [111]:
sel.get_support()

array([ True,  True,  True,  True, False,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
        True,  True,  True])

In [112]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

9

In [113]:
print(selected_feat)

Index(['nHM', 'F04', 'NssssC', 'nCb', 'F03', 'F02_CN', 'SM6_L', 'Mi',
       'SpPosA_B'],
      dtype='object')


In [117]:
X_train[X_train.columns[(sel.get_support())]]

,nHM,F04,NssssC,nCb,F03,F02_CN,SM6_L,Mi,SpPosA_B
0,0.0,0.0,0.0,0.0,0.0,0.0,-0.710938,0.328503,0.000615
1,0.0,0.0,0.0,0.0,0.0,0.0,-1.039247,0.423761,-1.207722
2,0.0,0.0,0.0,0.0,0.0,0.0,-0.583850,0.804797,-1.357207
3,0.0,0.0,0.0,0.0,0.0,0.0,-3.544517,1.519239,-2.204289
4,0.0,0.0,0.0,0.0,0.0,0.0,-0.091975,0.566650,-0.796638
...,...,...,...,...,...,...,...,...,...
3418,0.0,0.0,0.0,2.0,0.0,0.0,0.091671,-0.595750,0.194882
3419,0.0,0.0,0.0,3.0,2.0,2.0,-0.231644,-0.405399,0.719328
3420,0.0,0.0,0.0,2.0,0.0,0.0,0.211133,-0.909863,0.571714
3421,0.0,0.0,0.0,0.0,0.0,0.0,-1.513582,1.708718,-1.097402
